In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import keras as kr

In [2]:
# DF de entrenamiento, con todas las variables y la columna tier completa
df_train: pd.DataFrame = pd.DataFrame(columns=("data", "tier"))
df_train.loc[0] = [50000, 1]
df_train.loc[1] = [49000, 2]
df_train.loc[2] = [40000, 2]
df_train.loc[3] = [39000, 3]
df_train.loc[4] = [30000, 3]
df_train.loc[5] = [21000, 3]
df_train.loc[6] = [20000, 4]
df_train.loc[7] = [11000, 4]
df_train.loc[8] = [10000, 5]
df_train.loc[9] = [0, 5]


# DF de test, con todas las variables y la columna tier completa
df_test: pd.DataFrame = pd.DataFrame(columns=("data", "tier"))
df_test.loc[0] = [80000, 1]
df_test.loc[1] = [37000, 3]
df_test.loc[2] = [32000, 3]
df_test.loc[3] = [18000, 4]
df_test.loc[4] = [1000, 5]
df_test.loc[5] = [200, 5]
df_test.loc[6] = [45000, 2]

## Preparación de datos

In [3]:
# Separar características y etiquetas

X_train: pd.DataFrame = df_train.drop('tier', axis=1)
y_train: pd.Series = df_train['tier']

X_test: pd.DataFrame = df_test.drop('tier', axis=1)
y_test: pd.Series = df_test['tier']

# Normalizar los datos
scaler: StandardScaler = StandardScaler()
X_train: pd.DataFrame = scaler.fit_transform(X_train)
X_test: pd.DataFrame = scaler.transform(X_test)

# Convertir etiquetas a categorías
y_train = kr.utils.to_categorical(y_train - 1, num_classes=5)
y_test = kr.utils.to_categorical(y_test - 1, num_classes=5)

## Definición del modelo

In [4]:
# Definir el modelo
model: kr.models.Sequential = kr.models.Sequential([
    kr.layers.Input(shape=(X_train.shape[1],)),
    kr.layers.Dense(64, activation='relu'),
    kr.layers.Dense(32, activation='relu'),
    kr.layers.Dense(5, activation='softmax')
])

# Compilar el modelo
model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']
)

## Entrenamiento y evaluación del modelo

In [5]:
# Aquí te he hecho un entrenamiento con unos parámetros a ojo, pero es solo 
# una prueba.

# Lo suyo sería que entrenaras con distintos epochs, validation_splits y 
# batch_size para encontrar el "punto dulce" de precisión del modelo.

# Entrenar el modelo
history = model.fit(
    X_train, y_train, epochs=100, validation_split=0.05, batch_size=1
)

# Evaluar el modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')

Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.2547 - loss: 1.6208 - val_accuracy: 1.0000 - val_loss: 1.1563
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1347 - loss: 1.6167 - val_accuracy: 1.0000 - val_loss: 1.1343
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1060 - loss: 1.6981 - val_accuracy: 1.0000 - val_loss: 1.0988
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.4487 - loss: 1.4915 - val_accuracy: 1.0000 - val_loss: 1.0433
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3652 - loss: 1.5775 - val_accuracy: 1.0000 - val_loss: 1.0287
Epoch 6/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3944 - loss: 1.4421 - val_accuracy: 1.0000 - val_loss: 0.9770
Epoch 7/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5537 - loss: 1.3762 - val_accuracy: 1.0000 - val_loss: 0.9162
Epoch 8/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.3454 - loss: 1.4637 - val_accuracy: 1.0000 - val_loss: 0.8928

## Predicción de tiers

In [9]:
# DF con datos pero sin valor de tiers
df_score: pd.DataFrame = pd.DataFrame(columns=("data",))

df_score.loc[0] = [800000]
df_score.loc[1] = [45500]
df_score.loc[2] = [27900]
df_score.loc[3] = [12000]
df_score.loc[4] = [440]

In [10]:
# Normalizar los datos de df_score
X_score = df_score.values
X_score = scaler.transform(X_score)

# Predecir los tiers
predictions = model.predict(X_score)
predicted_tiers = predictions.argmax(axis=1) + 1

# Añadir las predicciones a df_score
df_score['predicted_tier'] = predicted_tiers


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


c:\Users\pablo\Documents\GitHub\tf_tiers\env\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [11]:
df_score

,data,predicted_tier
0,800000,1
1,45500,2
2,27900,3
3,12000,4
4,440,5
